### Running run-level frequency-based modelling from this notebook

In [5]:
%%time

experiment_id = "1_attention"
mri_id = "7T"
oscprep_dir = f"/data/{experiment_id}/{mri_id}/bids/derivatives/oscprep_grayords_fmapless"
out_dir = f"/scratch/{experiment_id}/{mri_id}/derivatives/run_level"
sub_id = '000'
ses_id = '20230801'
task_id = 'wbpilot'
run_id = '02'
search_frequencies = [.125, .2]
time_window = (14+25, 240) # second
smooth_mm = 0 # ONLY WORKS FOR CIFTI
image_type = "NIFTI"
#nordic_dir = f"/data/{experiment_id}/{mri_id}/bids/derivatives/nordic"
nordic_dir = "None"
#,'05','06','07','08','09','10','11','12','13','14','15','16','17'
for run_id in ['04','05','06','07','08','09','10','11','12','13','14','15','16','17']:
    !python3 /opt/app/scripts/fla.py \
        {experiment_id} \
        {mri_id} \
        {oscprep_dir} \
        {out_dir}_proc-NONORDIC_{image_type} \
        {sub_id} \
        {ses_id} \
        {task_id} \
        {run_id} \
        {' '.join([str(i) for i in search_frequencies])} \
        {' '.join([str(i) for i in time_window])} \
        --smooth-mm {smooth_mm} \
        --denoise-only \
        --nordic-dir {nordic_dir} \
        --image-type {image_type}

 Experiment ID: 1_attention
 MRI ID: 7T
 oscprep directory: /data/1_attention/7T/bids/derivatives/oscprep_grayords_fmapless
 output directory: /scratch/1_attention/7T/derivatives/run_level_proc-NONORDIC_NIFTI
 subject ID: 000
 session ID: 20230801
 task ID: wbpilot
 run ID: 04
 search frequencies: [0.125, 0.2] Hz
 time window: (39.0, 240.0) secs
 smooth (mm): 0
 denoise_only: True
 nordic_dir: None
 image_type: NIFTI


[00/01] Design matrix: min+motion24+wmcsf_mean+scrub
Skipping.
 Experiment ID: 1_attention
 MRI ID: 7T
 oscprep directory: /data/1_attention/7T/bids/derivatives/oscprep_grayords_fmapless
 output directory: /scratch/1_attention/7T/derivatives/run_level_proc-NONORDIC_NIFTI
 subject ID: 000
 session ID: 20230801
 task ID: wbpilot
 run ID: 05
 search frequencies: [0.125, 0.2] Hz
 time window: (39.0, 240.0) secs
 smooth (mm): 0
 denoise_only: True
 nordic_dir: None
 image_type: NIFTI


[00/01] Design matrix: min+motion24+wmcsf_mean+scrub
Skipping.
 Experiment ID: 1_attention


### Running run-level frequency-based modelling on ComputeCanada

In [ ]:
def search(base_dir, wildcard, error=True):
    search_path = Path(base_dir) / wildcard
    files = glob.glob(str(search_path))

    if not files:
        if error:
            raise FileNotFoundError(f"No files were found in: {search_path}")
        else:
            return []

    return files

from pathlib import Path
import itertools
import glob
import os
import nibabel as nib

from collections import defaultdict, Counter

# Output sbatch scripts here..
scripts_dir = "/data/scripts/03a_run_level"
scripts_dir = Path(scripts_dir)
if not scripts_dir.exists():
    scripts_dir.mkdir(parents=True)

# Specify projects
experiment_ids = ['1_frequency_tagging']
mri_ids = ['7T', '3T']
smooth_mm = 4 # specify surface smoothing - does not work when image-type == "NIFTI"
oscprep_dir = 'oscprep_grayords_fmapless' # specify oscprep preproc directory

for experiment_id, mri_id in itertools.product(experiment_ids, mri_ids):
    base_bids_dir = f'/data/{experiment_id}/{mri_id}/bids'
    sub_ids = [Path(i).stem for i in search(base_bids_dir, 'sub-*')]
    sub_ids.sort()

    oscprep_deriv_dir = f"{base_bids_dir}/derivatives/oscprep_grayords_fmapless"
    assert Path(oscprep_deriv_dir).exists()
    
    for sub_ix, sub_id in enumerate(sub_ids):
        sub_dir = f'{base_bids_dir}/{sub_id}'
        ses_ids = [Path(i).stem for i in search(sub_dir, 'ses-*')]
        ses_ids.sort()
        
        for ses_ix, ses_id in enumerate(ses_ids):
            ses_func_dir = f'{sub_dir}/{ses_id}/func'
            funcs = [Path(i).stem for i in search(ses_func_dir, '*part-mag_bold.nii.gz')]
            funcs.sort()

            vols_per_task_list = defaultdict(list)
            for func in funcs:
                nifti = Path(f"{ses_func_dir}/{func}.gz")
                task_id = func.split('task-')[1].split('_')[0]
                if task_id == 'wholebrain': continue
                n_vols = nib.load(nifti).shape[-1]
                vols_per_task_list[task_id].append(n_vols)

            vols_per_task = defaultdict(int)
            for k, v in vols_per_task_list.items():
                counter = Counter(v)
                vols_per_task[k] = counter.most_common(1)[0][0]

            for func in funcs:
                nifti = Path(f"{ses_func_dir}/{func}.gz")
                task_id = func.split('task-')[1].split('_')[0]
                if task_id == 'wholebrain': continue
                run_id = func.split('run-')[1].split('_')[0]
                n_vols = nib.load(nifti).shape[-1]
                inconsistent_vols = n_vols != vols_per_task[task_id]

                # Check if oscprep ran correctly
                oscprep_check_wildcards = [
                    f"{sub_id}_{ses_id}_task-{task_id}*run-{run_id}*space-T1w_boldref.nii.gz",
                    f"{sub_id}_{ses_id}_task-{task_id}*run-{run_id}*space-T1w_desc-boldref_brainmask.nii.gz",
                    f"{sub_id}_{ses_id}_task-{task_id}*run-{run_id}*space-T1w_desc-preproc_bold.nii.gz",
                    f"{sub_id}_{ses_id}_task-{task_id}*run-{run_id}*desc-preproc_bold.dtseries.nii",
                    f"{sub_id}_{ses_id}_task-{task_id}*run-{run_id}*desc-preproc_bold.json",
                    f"{sub_id}_{ses_id}_task-{task_id}*run-{run_id}*desc-confounds_timeseries.tsv",
                    f"{sub_id}_{ses_id}_task-{task_id}*run-{run_id}*desc-confounds_timeseries.json",
                ]

                preproc_flag = True
                preproc_paths = []
                for wildcard in oscprep_check_wildcards:
                    _path = search(f"{oscprep_deriv_dir}/bold_preproc/{sub_id}/{ses_id}/func", wildcard, error=False)
                    assert len(_path) in [0,1]
                    if len(_path) == 0:
                        preproc_flag = False
                    else:
                        preproc_paths += _path

                if preproc_flag and not inconsistent_vols:

                    if task_id.startswith(("localizer", "entrain", "control")):

                        experiment_search_frequencies = {
                            "localizer_7T": [.2],
                            "entrain_7T": [.2, .5],
                            "control_7T": [.2, .5],
                            "localizer_3T": [.125],
                            "entrain_3T": [.125, .2],
                            "control_3T": [.125, .2],
                        }

                        stim_start_time = 14
                        experiment_stim_end = {
                            "localizer_7T": stim_start_time+185,
                            "entrain_7T": stim_start_time+185,
                            "control_7T": stim_start_time+185,
                            "localizer_3T": stim_start_time+205,
                            "entrain_3T": stim_start_time+205,
                            "control_3T": stim_start_time+205,
                        }

                        k = f"{task_id.split('Q')[0]}_{mri_id}"
                        assert k in experiment_search_frequencies.keys()
                        search_frequencies = experiment_search_frequencies[k]
                        stim_end_time = experiment_stim_end[k]

                        for stim_delay_time in [25]:
                            
                            time_window = (stim_start_time+stim_delay_time, stim_end_time)

                            oscprep_dir = f"/data/{experiment_id}/{mri_id}/bids/derivatives/oscprep_grayords_fmapless"
                            out_dir = f"/scratch/{experiment_id}/{mri_id}/derivatives/run_level_s{smooth_mm}/truncate-{time_window[0]}-{time_window[1]}"
                            CONTAINER_DIR= '/project/def-mmur/gngo4/containers'

                            # Check to see if fla was run to completion by counting number of expected outputs
                            n_out = len(search(out_dir,f"*_experiment-*/{sub_id}/{ses_id}/task-{task_id}/run-{run_id}/GLM/*", error=False))
                            run_flag = (n_out==144) or (n_out==272) # These are expected output amount - manually change this depending on experiment
                            # Skip if ran already
                            if run_flag:
                                continue


                            all_run_dirs = f"/scratch/{experiment_id}/{mri_id}/derivatives/run_level_s{smooth_mm}/truncate-{time_window[0]}-{time_window[1]}/*_experiment-*/{sub_id}/{ses_id}/task-{task_id}/run-{run_id}"
                            #!rm -rf {all_run_dirs}
                            print(n_out)
                            
                        
                            print(
                                experiment_id, 
                                mri_id, 
                                sub_id.split('-')[-1],
                                ses_id.split('-')[-1], 
                                task_id, 
                                run_id, 
                                search_frequencies, 
                                time_window,
                            )

                            txt = f"""#!/bin/bash
module load apptainer/1.1

singularity run \\
    --bind /project/def-mmur/gngo4/data/fastfmri:/data \\
    --bind /scratch/gngo4:/scratch \\
    --bind /project/def-mmur/gngo4/projects/fastfmri_toolbox:/opt/app \\
    {CONTAINER_DIR}/neuroimaging-notebook-v2.simg \\
    python3 /opt/app/scripts/fla.py \\
    {experiment_id} \\
    {mri_id} \\
    {oscprep_dir} \\
    {out_dir} \\
    {sub_id.split('-')[-1]} \\
    {ses_id.split('-')[-1]} \\
    {task_id} \\
    {run_id} \\
    {' '.join([str(i) for i in search_frequencies])} \\
    {' '.join([str(i) for i in time_window])} \\
    --smooth-mm {smooth_mm}
                            """
                            cmd_path = f"{scripts_dir}/{experiment_id}_{mri_id}.s{smooth_mm}.trunc-{time_window[0]}-{time_window[1]}.{sub_id}_{ses_id}_task-{task_id}_run-{run_id}_fla.sh"
                            with open(cmd_path, 'w') as f:
                                f.write(txt)

In [ ]:
#!sbatch --time=00:45:00 --mem=32GB --account=def-mmur {cmd_path}